In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, Dropout, GaussianNoise
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import os


Using TensorFlow backend.


In [2]:
import numpy as np
import cv2
import pandas as pd
import os
import json

path = "D:/MaunaKea/TrainingSetImagesDir/"
patients = {}

# Creer un dict avec tous les paths des images par patient
for i in range(61):
    patients[str(i)] = []
    for file in os.listdir(path):
        if file.endswith("_" + str(i) + ".png"):
            patients[str(i)] += [file]

# creer un json pour recupérer le dict apres
with open('unsorted_patients.txt', 'w') as json_file:
    json.dump(patients, json_file)

targets = pd.read_csv("train_target.csv")
targets['patient'] = targets['image_filename'].map(lambda x: x.partition('_')[-1].partition('_')[-1].partition('.')[0])
targets = targets.sort_values(by = ['patient', 'image_filename']).set_index(keys = ['image_filename'])

In [3]:
keys = list(patients.keys())
for key in keys:
    if (patients[key] == []):
        patients.pop(key)

In [4]:
validation_set = targets.loc[patients['10'] + patients['8'] + patients['7'] + patients['6']]# + patients['1'] + patients['0']]
training_set = targets.drop(validation_set.index)

In [5]:
small_training_path = "D:/MaunaKea/TrainingSetImagesDir/"
small_training_images = np.zeros((training_set.shape[0], 224,224, 3))
small_training_classes = np.zeros(training_set.shape[0])

small_validation_path = "D:/MaunaKea/TrainingSetImagesDir/"
small_validation_images = np.zeros((validation_set.shape[0], 224, 224, 3))
small_validation_classes = np.zeros(validation_set.shape[0])

i = 0
for image_path in training_set.index:
    small_training_images[i] = cv2.resize(cv2.imread(small_training_path + image_path), (224, 224))
    #small_training_images[i] = preprocess_input(small_training_images[i])
    small_training_classes[i] = training_set.iloc[i]['class_number']
    i += 1
    
i = 0
for image_path in validation_set.index:
    small_validation_images[i] = cv2.resize(cv2.imread(small_validation_path + image_path), (224, 224))
    small_validation_images[i] = preprocess_input(small_validation_images[i])
    small_validation_classes[i] = validation_set.iloc[i]['class_number']
    i += 1
    


In [6]:
from keras.utils import to_categorical

small_training_classes = to_categorical(small_training_classes)
small_validation_classes = to_categorical(small_validation_classes)

In [23]:
from keras import regularizers
base_model = VGG16(include_top = False, input_shape = small_training_images.shape[1:], weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# input1 = Input(shape = small_training_images.shape[1:])
# x = GaussianNoise(0.05)(input1)
x = base_model(input1)
x = Flatten()(x)

x = Dense(1024, kernel_initializer = 'glorot_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(1024, kernel_initializer = 'glorot_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(4, kernel_initializer = 'glorot_normal', activation = 'softmax')(x)

model = Model(inputs = input1, outputs = x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              25691136  
_________________________________________________________________
batch_normalization_13 (Batc (None, 1024)              4096      
_________________________________________________________________
activation_13 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 1024)              0         
__________

In [24]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range = 180,
   #                          brightness_range = [0.8, 1.2],
                             width_shift_range = 30,
                             height_shift_range = 30,
                             zoom_range = [1.0, 1.2],
                             preprocessing_function = preprocess_input,
                             horizontal_flip = True,
                             vertical_flip = True)
#datagen.fit(small_training_images)
#sdds = datagen.standardize(small_validation_images)

In [25]:
lr = 1e-4
batch_size = 32
n_epochs = 20

optimizer = Adam(lr = lr)
lr_reducer = ReduceLROnPlateau(factor=0.1,
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-7)

callbacks = [lr_reducer]
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
history = model.fit_generator(datagen.flow(small_training_images, small_training_classes, batch_size),
                                  steps_per_epoch = small_training_images.shape[0]/(batch_size * 2), epochs = n_epochs,
                                  validation_data = (small_validation_images, small_validation_classes),
                                  callbacks = callbacks)

Epoch 1/20
133/132 [==============================] - 54s 403ms/step - loss: 0.9887 - acc: 0.6245 - val_loss: 1.1627 - val_acc: 0.4133
Epoch 2/20
133/132 [==============================] - 49s 371ms/step - loss: 0.6909 - acc: 0.7376 - val_loss: 0.5717 - val_acc: 0.7487
Epoch 3/20
133/132 [==============================] - 50s 372ms/step - loss: 0.5837 - acc: 0.7827 - val_loss: 0.5840 - val_acc: 0.7497
Epoch 4/20
133/132 [==============================] - 49s 370ms/step - loss: 0.5262 - acc: 0.8050 - val_loss: 0.5098 - val_acc: 0.7928
Epoch 5/20
133/132 [==============================] - 49s 372ms/step - loss: 0.4694 - acc: 0.8266 - val_loss: 0.6564 - val_acc: 0.7169
Epoch 6/20
133/132 [==============================] - 49s 368ms/step - loss: 0.4680 - acc: 0.8346 - val_loss: 0.5256 - val_acc: 0.7826
Epoch 7/20
133/132 [==============================] - 50s 374ms/step - loss: 0.4149 - acc: 0.8461 - val_loss: 0.7045 - val_acc: 0.7077
Epoch 8/20
133/132 [==============================] - 5

In [37]:
model.save("vgg16_augmented_nornal_images_no_denoised_85_val_acc.h5")

In [ ]:
small_training_path = "D:/MaunaKea/Train/"
small_training_images = np.zeros((training_set.shape[0], 224, 224, 3))
small_training_classes = np.zeros(training_set.shape[0])

small_validation_path = "D:/MaunaKea/Train/"
small_validation_images = np.zeros((validation_set.shape[0], 224, 224, 3))
small_validation_classes = np.zeros(validation_set.shape[0])

i = 0
for image_path in training_set.index:
    small_training_images[i] = cv2.resize(cv2.imread(small_training_path + image_path), (224, 224))
    small_training_images[i] = preprocess_input(small_training_images[i])
    small_training_classes[i] = training_set.iloc[i]['class_number']
    i += 1
    
i = 0
for image_path in validation_set.index:
    small_validation_images[i] = cv2.resize(cv2.imread(small_validation_path + image_path), (224,224))
    small_validation_images[i] = preprocess_input(small_validation_images[i])
    small_validation_classes[i] = validation_set.iloc[i]['class_number']
    i += 1
    